# Mathematical Underpinnings - Lab 6

In [1]:
from sklearn.metrics import mutual_info_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from tqdm import tqdm

## Useful functions

In [2]:
def discetize_2bins(X):
    X_discrete = 1*(X >= 0)
    return X_discrete

In [3]:
def conditional_permutation(X, Z):

    z_values = np.unique(Z)
    n_z_values = len(z_values)
    n = len(Z)

    X_b = np.zeros(n)

    for i in range(n_z_values):

        z_value_tmp = z_values[i]

        X_b[Z == z_value_tmp] = np.random.permutation(X[Z == z_value_tmp])

    return X_b

In [4]:
def conditional_mutual_information(X, Y, Z):
    z_values = np.unique(Z)
    n_z_values = len(z_values)
    n = len(Z)

    cmi = 0

    for i in range(n_z_values):

        z_value_tmp = z_values[i]
        z_condition = (Z == z_value_tmp)

        X_z = X[z_condition]
        Y_z = Y[z_condition]

        mi_XY_z = mutual_info_score(X_z, Y_z)
        p_z = np.sum(z_condition)/n

        cmi += p_z*mi_XY_z

    return cmi

In [5]:
# II(X;Y;Z)
def interaction_information(X, Y, Z):
    return conditional_mutual_information(X, Y, Z) - mutual_info_score(X, Y)

In [6]:
# II(X;Y;Z1;Z2)
def interaction_information2(X, Y, Z1, Z2):
    Z_1_and_2 = 2*Z2 + Z1
    return interaction_information(X, Y, Z_1_and_2) - interaction_information(X, Y, Z1) - interaction_information(X, Y, Z2)

## Task 1

In [7]:
def secmi2(X, Y, Z):
    n_col_Z = Z.shape[1]
    Z_1dim = np.dot(Z, 2**np.linspace(0, n_col_Z-1, n_col_Z))
    return mutual_info_score(X, Y) + conditional_mutual_information(X, Y, Z_1dim)

def secmi3(X, Y, Z):
    n_col_Z = Z.shape[1]
    Z_1dim = np.dot(Z, 2**np.linspace(0, n_col_Z-1, n_col_Z))
    secmi = mutual_info_score(X, Y) + conditional_mutual_information(X, Y, Z_1dim)
    for i in range(n_col_Z-1):
        for j in range(i+1, n_col_Z):
            secmi += interaction_information2(X, Y, Z[:, i], Z[:, j])
    return secmi

### a)

In [8]:
def cond_indep_test_permutation(X, Y, Z, B, stat):

    n_col_Z = Z.shape[1]
    Z_1dim = np.dot(Z, 2**np.linspace(0, n_col_Z-1, n_col_Z))

    if stat == "cmi":
        stat_value = conditional_mutual_information(X, Y, Z_1dim)
    if stat == "secmi2":
        stat_value = secmi2(X, Y, Z)
    if stat == "secmi3":
        stat_value = secmi3(X, Y, Z)

    condition_p_value = 0
    for b in range(B):
        X_b = conditional_permutation(X, Z_1dim)

        if stat == "cmi":
            stat_value_b = conditional_mutual_information(X_b, Y, Z_1dim)
        if stat == "secmi2":
            stat_value_b = secmi2(X_b, Y, Z)
        if stat == "secmi3":
            stat_value_b = secmi3(X_b, Y, Z)

        if stat_value <= stat_value_b:
            condition_p_value += 1

    p_value = (1 + condition_p_value)/(1 + B)

    return 2*len(X)*stat_value, p_value

### b)

In [18]:
def discretize(v):
    v[v < 0] = -1
    v[v >= 0] = 1
    return v

def sample_from_model(n=100):
    Y = discretize(np.random.normal(0, 1, n))
    Z1 = discretize(np.random.normal(Y/2, 1))
    Z2 = discretize(np.random.normal(Y/2, 1))
    Z3 = discretize(np.random.normal(Y/2, 1))
    X = discretize(np.random.normal(Z1/2, 1))
    Z = np.concatenate([np.expand_dims(Z1, 1), np.expand_dims(Z2, 1), np.expand_dims(Z3, 1)], axis=1)
    return X, Y, Z

X, Y, Z = sample_from_model(1000)

In [15]:
def run_test(X, Y, Z, test_type, N=100):
    stat_values=[]
    p_values = []
    for i in range(N):
        stat_value, p_value = cond_indep_test_permutation(X, Y, Z, 50, test_type)
        stat_values.append(stat_value)
        p_values.append(p_value)
    rejected = np.sum(np.array(p_values)<=0.05)
    print(f'average statistic value: {np.mean(stat_values)}, average p-value: {np.mean(p_values)}')
    print(f'{rejected}/{N} hypothesis rejected')
    
def run_all_tests(X, Y, Z, N=100):
    print(f'Test of conditional independence with conditional mutual information:')
    run_test(X, Y, Z, 'cmi', N=100)
    print('----------------------------------------------------')
    print(f'Test of conditional independence with SECMI2:')
    run_test(X, Y, Z, 'secmi2', N=100)
    print('----------------------------------------------------')
    print(f'Test of conditional independence with SECMI3:')
    run_test(X, Y, Z, 'secmi3', N=100)

$H_{0}: X \!\perp\!\!\!\perp Y | (Z_{1}, Z_{2})$

In [16]:
run_all_tests(X, Y, Z[:, :2], N=100)

Test of conditional independence with conditional mutual information:
average statistic value: 1.7790159370889287, average p-value: 0.789607843137255
0/100 hypothesis rejected
----------------------------------------------------
Test of conditional independence with SECMI2:
average statistic value: 26.932198810337226, average p-value: 0.24588235294117644
0/100 hypothesis rejected
----------------------------------------------------
Test of conditional independence with SECMI3:
average statistic value: 28.098418638571747, average p-value: 0.3227450980392157
0/100 hypothesis rejected


$H_{0}: X \!\perp\!\!\!\perp Y | (Z_{2}, Z_{3})$

In [17]:
run_all_tests(X, Y, Z[:, 1:], N=100)

Test of conditional independence with conditional mutual information:
average statistic value: 20.25994844049766, average p-value: 0.020784313725490187
100/100 hypothesis rejected
----------------------------------------------------
Test of conditional independence with SECMI2:
average statistic value: 45.413131313745964, average p-value: 0.019607843137254895
100/100 hypothesis rejected
----------------------------------------------------
Test of conditional independence with SECMI3:
average statistic value: 46.66184189683528, average p-value: 0.019607843137254895
100/100 hypothesis rejected


### c)

In [53]:
def sample_from_model2(n=1000):
    X = np.random.binomial(1, 1/2, n)
    Z1 = np.random.binomial(1, 1/2, n)
    Z2 = np.random.binomial(1, 1/2, n)
    Z3 = np.random.binomial(1, 1/2, n)
    Z = np.concatenate([np.expand_dims(Z1, 1), np.expand_dims(Z2, 1), np.expand_dims(Z3, 1)], axis=1)
    
    cond = (X + Z1 + Z2) % 2
    f = lambda x: x + 0.2 * (-1)**x
    prob = f(cond)
    Y = np.random.binomial(1, prob)
    return X, Y, Z

X, Y, Z = sample_from_model2()

$H_{0}: X \!\perp\!\!\!\perp Y | (Z_{1}, Z_{2})$

In [54]:
run_all_tests(X, Y, Z[:, :2], N=100)

Test of conditional independence with conditional mutual information:
average statistic value: 361.6381442513802, average p-value: 0.019607843137254895
100/100 hypothesis rejected
----------------------------------------------------
Test of conditional independence with SECMI2:
average statistic value: 361.74161934665045, average p-value: 0.019607843137254895
100/100 hypothesis rejected
----------------------------------------------------
Test of conditional independence with SECMI3:
average statistic value: 720.6979523455254, average p-value: 0.019607843137254895
100/100 hypothesis rejected


$H_{0}: X \!\perp\!\!\!\perp Y | (Z_{2}, Z_{3})$

In [55]:
run_all_tests(X, Y, Z[:, 1:], N=100)

Test of conditional independence with conditional mutual information:
average statistic value: 4.128131518255773, average p-value: 0.40470588235294125
0/100 hypothesis rejected
----------------------------------------------------
Test of conditional independence with SECMI2:
average statistic value: 4.231606613526196, average p-value: 0.503921568627451
0/100 hypothesis rejected
----------------------------------------------------
Test of conditional independence with SECMI3:
average statistic value: 5.678436867287572, average p-value: 0.43568627450980396
0/100 hypothesis rejected


## Task 2
 
in R